# 檢查運算資源

In [445]:
import psutil

# Check RAM
ram = psutil.virtual_memory()
print(f'Total RAM: {ram.total / (1024 ** 3):.2f} GB')
print(f'Available RAM: {ram.available / (1024 ** 3):.2f} GB')

# Check Disk Space
disk = psutil.disk_usage('/')
print(f'Total Disk Space: {disk.total / (1024 ** 3):.2f} GB')
print(f'Used Disk Space: {disk.used / (1024 ** 3):.2f} GB')
print(f'Free Disk Space: {disk.free / (1024 ** 3):.2f} GB')

Total RAM: 52.96 GB
Available RAM: 47.79 GB
Total Disk Space: 201.23 GB
Used Disk Space: 27.70 GB
Free Disk Space: 173.51 GB


In [446]:
# from google.colab import drive
# drive.mount('/content/drive')

# 載入套件

In [447]:
# Importing necessary libraries
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torchvision.models as models

from sklearn.ensemble import VotingClassifier, BaggingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier

from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.utils.validation import has_fit_parameter
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

# 上傳資料

In [448]:
# 讀取資料
def load_data(file_path):
    try:
      df = pd.read_csv('/content/sample_data/df_final_with_eco.csv', encoding='utf-8')
      return df
    except Exception as e:
      print(f"Error loading data: {e}")
      return None


In [449]:
# import pandas as pd

# # 使用相對路徑載入資料
# df = pd.read_csv('/content/sample_data/df_final_with_eco.csv', encoding='utf-8')

In [450]:
# df.head(2)

# 資料清整 (這裡給可靠的歐爸了)

> 這邊主要是依據main()出現問題來添加的

> def split_datasets(df):

> 大致切割一下資料

- 001-016 實價登入 estate

- 017-052 經濟指標 eco

- 053-156 比鄰   near

In [451]:
# # 切割出經濟指標資料集 (017-052行)
# df_eco = df.iloc[16:52]
# # 切割出其餘資料集 (001-016行 實價登入 和 053-156行 比鄰)
# df_near_estate = pd.concat([df.iloc[:16], df.iloc[52:]], ignore_index=True)

In [452]:
# 切割資料集
def split_datasets(df):
    # 切割出經濟指標資料集 (017-052行)
    df_eco = df.iloc[16:52]
    # 切割出其餘資料集 (001-016行 實價登入 和 053-156行 比鄰)
    df_near_estate = pd.concat([df.iloc[:16], df.iloc[52:]], ignore_index=True)
    return df_eco, df_near_estate


In [453]:
# 處理缺失值
def handle_missing_values(X):
    imputer = SimpleImputer(strategy='mean')
    X_imputed = imputer.fit_transform(X)
    return X_imputed

# 設定目標和特徵值

> def prepare_data(df_near_estate):

> df_near_estate

In [454]:
# # 設置 new_per_ping 為 y，其餘行數為 X
def prepare_data(df_near_estate):
    y = df_near_estate['new_per_ping']
    X = df_near_estate.drop(columns=['new_per_ping'])
    # 將連續變量轉換為分類變量
    y = pd.cut(y, bins=10, labels=False)  # 將數據分成 10 個區間
    # 確保標籤是連續的整數
    y, _ = pd.factorize(y)
    return X, y

In [455]:
# # 設置 new_per_ping 為 y，其餘行數為 X
# def prepare_data(df_near_estate):
#     y = df_near_estate['new_per_ping']
#     X = df_near_estate.drop(columns=['new_per_ping'])
#     # 將連續變量轉換為分類變量
#     y = pd.cut(y, bins=10, labels=False)  # 將數據分成 5 個區間
#     return X, y

# # 如果 new_per_ping 是連續變量，這種分箱方法將會將其轉換為離散的分類標籤。

In [456]:
# y = df_near_estate['new_per_ping']
# X = df_near_estate.drop(columns=['new_per_ping'])

# 切割資料為訓練、測試、(驗證)

> 這個部分會在主程式中執行

> df_near_estate 資料進行切割，使其分成 70% 的訓練集、15% 的測試集和 15% 的驗證集

> df_near_estate

> 設置 X_train, y_train 和 X_test, y_test

> 驗證集的eco集5-6月資料我明天會抓

In [457]:
'''
# 取得資料的總筆數
n = len(df_near_estate)

# 設定訓練集為總筆數的80%、測試集為總筆數的20%
n_train = int(0.8 * n)
n_test = n - n_train

# 將資料前80%的資料作為訓練集
X_train = X[:n_train].values
y_train = y[:n_train].values

# 將資料最後20%的資料作為測試集
X_test = X[-n_test:].values
y_test = y[-n_test:].values

# 明天蒐集 5-6 月經濟指標作為驗證集

'''

'\n# 取得資料的總筆數\nn = len(df_near_estate)\n\n# 設定訓練集為總筆數的80%、測試集為總筆數的20%\nn_train = int(0.8 * n)\nn_test = n - n_train\n\n# 將資料前80%的資料作為訓練集\nX_train = X[:n_train].values\ny_train = y[:n_train].values\n\n# 將資料最後20%的資料作為測試集\nX_test = X[-n_test:].values\ny_test = y[-n_test:].values\n\n# 明天蒐集 5-6 月經濟指標作為驗證集\n\n'

# 模型建立 - Bagging Classifier 的打包

- **使用 BaggingClassifier 對五種不同的分類器（決策樹、KNN、XGBoost、神經網絡和SVM）進行封裝。**

 - create_bagging_classifiers 這個函數，創建包含多種袋裝分類器的字典。

 - classifiers 是一個字典，包含不同的基本分類器。

 - 返回一個字典，其中每個分類器都被袋裝技術包裝。


- **Bagging Classifier 的參數介紹**
  - base_estimator 是指定的基本分類器

  - n_estimators 是袋裝中使用的基分類器的數量，
   - 設置 基分類器數量 為 10

  - random_state 用於控制隨機數生成器，以保證結果的可重現性。

   - 設置 隨機種子 為 42

In [458]:
def create_bagging_classifiers():
    classifiers = {
        'KNN': SampleWeightWrapper(KNeighborsClassifier()),
        'XGBoost': SampleWeightWrapper(XGBClassifier()),
        'Neural Network': SampleWeightWrapper(MLPClassifier()),
        'SVM': SampleWeightWrapper(SVC(probability=True, max_iter=10000)),
        'Random Forest': SampleWeightWrapper(RandomForestClassifier())
        #'Custom NN': MyModel()  # 使用自定義的神經網絡模型
    }

    bagging_classifiers = {}
    for name, clf in classifiers.items():
        bagging_classifiers[name] = BaggingClassifier(
            estimator=clf,
            n_estimators=10,
            random_state=42,
            bootstrap=True, # 使用bootstrap進行樣本選擇
            bootstrap_features=False # 不使用特徵bootstrap
        )

    return bagging_classifiers, classifiers

## 自定義 - Custom NN

> 在『自定義初始化基本分類器』當中使用的 function

> the base classifiers

> 基分類器 (中國)

- 觀察權重調整前、後的
 - bias
 - weights

In [459]:
class MyModel(BaseEstimator, ClassifierMixin):
    def __init__(self):
      super(MyModel, self).__init__()
      self.net = models.resnet18(weights=None)
      self.net.fc = nn.Linear(self.net.fc.in_features, 3)
      self.optimizer = torch.optim.Adam(self.net.parameters(), lr=0.001)
      self.criterion = nn.CrossEntropyLoss()

    def fit(self, X, y, sample_weight=None):
      self.net.train()
      X_tensor = torch.tensor(X, dtype=torch.float32)
      y_tensor = torch.tensor(y, dtype=torch.long)
      if sample_weight is not None:
        ample_weight_tensor = torch.tensor(sample_weight, dtype=torch.float32)
      else:
        sample_weight_tensor = torch.ones_like(y_tensor, dtype=torch.float32)

      for epoch in range(10):
        self.optimizer.zero_grad()
        outputs = self.net(X_tensor)
        loss = self.criterion(outputs, y_tensor)
        weighted_loss = loss * sample_weight_tensor
        weighted_loss.mean().backward()
        self.optimizer.step()
      return self

    def predict(self, X):
      self.net.eval()
      X_tensor = torch.tensor(X, dtype=torch.float32)
      outputs = self.net(X_tensor)
      _, predicted = torch.max(outputs, 1)
      return predicted.numpy()


In [460]:
# # Define the custom neural network model
# class MyModel(nn.Module):
#     def __init__(self):
#         super(MyModel, self).__init__()
#         self.net = models.resnet18(pretrained=False)
#         self.net.fc = nn.Linear(self.net.fc.in_features, 3)  # 修改輸出層大小以匹配Iris數據集

#         # 打印初始化權重和偏差
#         print('Initial weights: ', self.net.fc.weight[0][:10])
#         print('Initial bias: ', self.net.fc.bias[:10])

#         # 計算初始化權重和偏差的變異數
#         initial_weights_variance = self.calculate_variance(self.net.fc.weight)
#         initial_bias_variance = self.calculate_variance(self.net.fc.bias)
#         print('Initial weights variance:', initial_weights_variance)
#         print('Initial bias variance:', initial_bias_variance)
#         print('=====================')

#         # 自定義權重和偏差
#         self.net.fc.weight = torch.nn.Parameter(torch.ones(self.net.fc.weight.shape) * 0.9, requires_grad=True)
#         self.net.fc.bias = torch.nn.Parameter(torch.zeros(self.net.fc.bias.shape), requires_grad=True)

#         # 打印自定義後的權重和偏差
#         print('Custom weights: ', self.net.fc.weight[0][:10])
#         print('Custom bias: ', self.net.fc.bias[:10])

#         # 計算自定義後權重和偏差的變異數
#         custom_weights_variance = self.calculate_variance(self.net.fc.weight)
#         custom_bias_variance = self.calculate_variance(self.net.fc.bias)
#         print('Custom weights variance:', custom_weights_variance)
#         print('Custom bias variance:', custom_bias_variance)

#     def forward(self, x):
#         output = self.net(x)
#         return output

#     def calculate_variance(self, tensor):
#         return torch.var(tensor).item()

# 自定義 - 分類器包裝器

> 這個類用於包裝不支持 sample_weight 參數的分類器，以便能夠忽略 sample_weight 參數。。

In [461]:
# 自定義 - 分類器包裝器
class SampleWeightWrapper(BaseEstimator, ClassifierMixin):
    def __init__(self, estimator):
        self.estimator = estimator

    # 修改的部分：確保忽略不接受 sample_weight 的分類器
    def fit(self, X, y, sample_weight=None):
        if sample_weight is not None:
            try:
                self.estimator.fit(X, y, sample_weight=sample_weight)
            except TypeError:
                self.estimator.fit(X, y)
        else:
            self.estimator.fit(X, y)
        return self

    def predict(self, X):
        return self.estimator.predict(X)

    def predict_proba(self, X):
        if hasattr(self.estimator, "predict_proba"):
            return self.estimator.predict_proba(X)
        else:
            raise AttributeError(f"{self.estimator.__class__.__name__} does not support predict_proba method")

    def __getattr__(self, attr):
        return getattr(self.estimator, attr)

# 訓練 Bagging Classifiers & 增加樣本權重

- train_bagging_classifiers 這個函數，訓練每個袋裝分類器。

 - bagging_classifiers 是 create_bagging_classifiers 返回的袋裝分類器字典。

 - X_train 和 y_train 是 訓練數據 (train data)。

 - 返回訓練好的袋裝分類器字典。

In [462]:
def train_bagging_classifiers(bagging_classifiers, X_train, y_train):
    for name, classifier in bagging_classifiers.items():
        print(f"Training {name}")
        classifier.fit(X_train, y_train)
    return bagging_classifiers

In [463]:
# # Train the BaggingClassifiers
# bagging_decision_tree.fit(X_train, y_train)
# bagging_knn.fit(X_train, y_train)
# bagging_xgboost.fit(X_train, y_train)
# bagging_nn.fit(X_train, y_train)
# bagging_svm.fit(X_train, y_train)

# 用訓練好的 bagging classifiers 預測

- make_predictions 這個函數，使用訓練好的袋裝分類器 (bagging classifiers) 對測試集 (test data) 進行預測。

 - bagging_classifiers 是訓練好的袋裝分類器字典。

 - X_test 是測試數據。

 - 返回包含每個分類器預測結果的字典。

In [464]:
# make_predictions 函數
def make_predictions(bagging_classifiers, X_test, y_test):
    predictions = {}
    for name, classifier in bagging_classifiers.items():
        # 確保所有分類器具有相同的類別數量
        if hasattr(classifier, "n_classes_") and classifier.n_classes_ != len(np.unique(y_test)):
            continue
        # 進行預測並保存結果
        predictions[name] = classifier.predict(X_test)
    return predictions

In [465]:
# # Make predictions on the test set
# # predict 方法將測試數據（X_test）傳遞給每個袋裝分類器，並返回預測標籤。
# y_pred_dt = bagging_decision_tree.predict(X_test)
# y_pred_knn = bagging_knn.predict(X_test)
# y_pred_xgb = bagging_xgboost.predict(X_test)
# y_pred_nn = bagging_nn.predict(X_test)
# y_pred_svm = bagging_svm.predict(X_test)

# 計算每個袋裝分類器 (bagging classifiers) 的準確率

- calculate_accuracies 這個函數，對測試集 (test data) 的真實標籤（y_test）和預測標籤（y_pred）進行比較，並且計算每個袋裝分類器的準確率 (accuracy)。

 - predictions 是包含每個分類器預測結果的字典。

 - y_test 是測試數據的真實標籤。

 - 返回包含每個分類器準確率的字典。



In [466]:
# 計算每個袋裝分類器的準確率
def calculate_accuracies(predictions, y_test):
    accuracies = {}
    for name, y_pred in predictions.items():
        # 計算準確率並保存結果
        accuracies[name] = accuracy_score(y_test, y_pred)
    return accuracies

# 輸出每個袋裝分類器 (bagging classifiers) 的準確率

- print_accuracies 這個函數，輸出每個袋裝分類器的準確率。

 - accuracies 是包含每個分類器準確率的字典。

In [467]:
# 打印每個袋裝分類器的準確率
def print_accuracies(accuracies):
    for name, accuracy in accuracies.items():
        print(f"{name} Bagging Accuracy: {accuracy}")

# 使用例子

> (練習用的，可以省略)

In [468]:
'''
# 使用例子
classifiers = {
    'Decision Tree': DecisionTreeClassifier(),
    'KNN': KNeighborsClassifier(),
    'XGBoost': XGBClassifier(),
    'Neural Network': MLPClassifier(),
    'SVM': SVC()
}

# 假設 X_train, y_train, X_test, y_test 已經定義
# 你需要根據你的資料來定義這些變數
# X_train, y_train, X_test, y_test = ...

# 對少數類別樣本給予更高的權重
sample_weight = np.ones(len(y_train))
sample_weight[y_train == 1] *= 10

# 創建並訓練袋裝分類器
bagging_classifiers = create_bagging_classifiers(classifiers)
trained_classifiers = train_bagging_classifiers(bagging_classifiers, X_train, y_train, sample_weight)
# 使用訓練好的袋裝分類器進行預測
predictions = make_predictions(trained_classifiers, X_test)
# 計算準確率
accuracies = calculate_accuracies(predictions, y_test)
# 打印準確率
print_accuracies(accuracies)

'''

"\n# 使用例子\nclassifiers = {\n    'Decision Tree': DecisionTreeClassifier(),\n    'KNN': KNeighborsClassifier(),\n    'XGBoost': XGBClassifier(),\n    'Neural Network': MLPClassifier(),\n    'SVM': SVC()\n}\n\n# 假設 X_train, y_train, X_test, y_test 已經定義\n# 你需要根據你的資料來定義這些變數\n# X_train, y_train, X_test, y_test = ...\n\n# 對少數類別樣本給予更高的權重\nsample_weight = np.ones(len(y_train))\nsample_weight[y_train == 1] *= 10\n\n# 創建並訓練袋裝分類器\nbagging_classifiers = create_bagging_classifiers(classifiers)\ntrained_classifiers = train_bagging_classifiers(bagging_classifiers, X_train, y_train, sample_weight)\n# 使用訓練好的袋裝分類器進行預測\npredictions = make_predictions(trained_classifiers, X_test)\n# 計算準確率\naccuracies = calculate_accuracies(predictions, y_test)\n# 打印準確率\nprint_accuracies(accuracies)\n\n"

# 投票分類器 (voting_classifier) 和 袋裝分類器 (bagging_classifier) 的差異 (純敘述)

> **這兩個函數的主要區別在於它們所創建的分類器的種類及其運作方式：**

> **create_voting_classifier 函數：**

- 目的：創建一個投票分類器（Voting Classifier）。

- 機制：投票分類器會整合多個基分類器的預測結果，並通過投票（可以是硬投票或軟投票）來決定最終的預測結果。硬投票是根據每個基分類器的預測類別進行投票，而軟投票則是根據每個基分類器的預測概率進行投票。

- 組成：該函數使用 VotingClassifier 來組合五個基分類器（Decision Tree、KNN、XGBoost、NN 和 SVC），並使用軟投票機制來進行預測。


> **create_bagging_classifier 函數：**

- 目的：創建一個袋裝分類器（Bagging Classifier），其基分類器為投票分類器。

- 機制：袋裝分類器是一種集成方法，通過在訓練數據的不同子集上訓練多個基分類器來提高模型的穩定性和準確性。這些基分類器的預測結果將進行平均或投票以生成最終預測。袋裝分類器通常使用的是相同類型的基分類器，但這裡是用投票分類器作為基分類器。

- 組成：該函數使用 BaggingClassifier，將投票分類器作為其基分類器（base_estimator），並創建 50 個這樣的投票分類器來進行袋裝。

> **簡單總結：**

- create_voting_classifier：創建一個單一的投票分類器，整合多個基分類器的預測結果，通過投票來決定最終的預測結果。

- create_bagging_classifier：創建一個袋裝分類器，使用多個投票分類器作為基分類器，通過在不同子集上訓練這些基分類器來提高模型的穩定性和準確性，最終的預測結果是這些投票分類器的平均或投票結果。

> **這兩個函數所創建的分類器的運作方式如下圖所示：**

- 投票分類器（Voting Classifier）：

 - 基分類器 1 → 預測
 - 基分類器 2 → 預測
 - 基分類器 3 → 預測
 - 基分類器 4 → 預測
 - 基分類器 5 → 預測
 - 投票（硬/軟） → 最終預測

- 袋裝分類器（Bagging Classifier）：

 - 投票分類器 1（包含多個基分類器） → 預測
 - 投票分類器 2（包含多個基分類器） → 預測
 - 投票分類器 3（包含多個基分類器） → 預測
 - ...
 - 投票分類器 50（包含多個基分類器） → 預測
 - 平均/投票 → 最終預測

# 創建投票分類器 (voting classifier) 的函數

In [469]:
def create_voting_classifier(base_clfs, X_train):
    print("Available classifiers in base_clfs:", base_clfs.keys())
    # 檢查所有基分類器是否支持 predict_proba 方法
    valid_estimators = []
    for name, clf in base_clfs.items():
        if hasattr(clf, "predict_proba"):
            # 檢查分類器的 predict_proba 方法返回的輸出形狀
            try:
                test_proba = clf.predict_proba(X_train[:10])
                print(f"{name} predict_proba shape: {test_proba.shape}")
                valid_estimators.append((name, clf))
            except Exception as e:
                print(f"{name} failed predict_proba check: {e}")

    if len(valid_estimators) == 0:
        raise ValueError("None of the classifiers support predict_proba method.")

    # 創建一個軟投票分類器
    voting_clf = VotingClassifier(estimators=valid_estimators, voting='soft')
    return voting_clf

# 創建袋裝投票分類器 (bagging classifier) 的函數

In [470]:
# 創建袋裝投票分類器的函數
def create_bagging_classifier(voting_clf):
    # 使用投票分類器作為基分類器創建袋裝分類器
    bagging_clf = BaggingClassifier(estimator=voting_clf, n_estimators=50, random_state=42)
    return bagging_clf

# 驗證袋裝投票分類器的函數

> 這裡會用到 驗證集

> 訓練並評估袋裝投票分類器的函數，增加樣本權重

In [ ]:
def train_and_evaluate_bagging_clf(bagging_clf, X_train, y_train, X_val, y_val, X_test, y_test):
    skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    val_accuracies = []

    for train_index, val_index in skf.split(X_train, y_train):
        X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
        y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

        if has_fit_parameter(bagging_clf, "sample_weight"):
            bagging_clf.fit(X_train_fold, y_train_fold, sample_weight=np.ones(len(y_train_fold)))
        else:
            bagging_clf.fit(X_train_fold, y_train_fold)

        # 確保 VotingClassifier 中的所有估計器具有相同的類別，並處理潛在的形狀不匹配問題
        unique_classes = np.unique(y_train)
        for clf in bagging_clf.estimators_:
            if hasattr(clf, "classes_"):
                clf.classes_ = unique_classes
            # 檢查估計器是否具有 'predict_proba' 方法，並處理潛在的形狀問題
            if hasattr(clf, "predict_proba"):
                try:
                    # 嘗試在小樣本上進行預測以檢查形狀錯誤
                    proba = clf.predict_proba(X_val_fold[:1])
                    if proba.shape[1] != len(unique_classes): # 檢查預測的類別數量是否匹配
                        print(f"估計器 {clf.__class__.__name__} 預測的類別數量不同。正在重新訓練完整的資料...")
                        clf.fit(X_train, y_train) # 在完整的訓練資料上重新訓練以確保一致性
                except ValueError as e:
                    print(f"估計器 {clf.__class__.__name__} 引發 ValueError: {e}。正在重新訓練完整的資料...")
                    clf.fit(X_train, y_train) # 在完整的訓練資料上重新訓練以潛在地解決問題

        val_predictions = bagging_clf.predict(X_val_fold)
        val_accuracy = accuracy_score(y_val_fold, val_predictions)
        val_accuracies.append(val_accuracy)

    val_accuracy = np.mean(val_accuracies)

    # 在對測試資料進行預測之前，確保 VotingClassifier 中的所有估計器具有相同的類別
    for clf in bagging_clf.estimators_:
        if hasattr(clf, "classes_"):
            clf.classes_ = unique_classes

    test_predictions = bagging_clf.predict(X_test)
    test_accuracy = accuracy_score(y_test, test_predictions)

    return val_accuracy, test_accuracy

In [471]:
# # ChatGPT-4o
# # 訓練並評估袋裝投票分類器
# def train_and_evaluate_bagging_clf(bagging_clf, X_train, y_train, X_val, y_val, X_test, y_test):
#     skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
#     val_accuracies = []

#     for train_index, val_index in skf.split(X_train, y_train):
#         X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
#         y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

#         if has_fit_parameter(bagging_clf, "sample_weight"):
#             bagging_clf.fit(X_train_fold, y_train_fold, sample_weight=np.ones(len(y_train_fold)))
#         else:
#             bagging_clf.fit(X_train_fold, y_train_fold)

#         unique_classes = np.unique(y_train)
#         for clf in bagging_clf.estimators_:
#             if hasattr(clf, "classes_"):
#                 clf.classes_ = unique_classes

#         val_predictions = bagging_clf.predict(X_val_fold)
#         val_accuracy = accuracy_score(y_val_fold, val_predictions)
#         val_accuracies.append(val_accuracy)

#     val_accuracy = np.mean(val_accuracies)
#     test_predictions = bagging_clf.predict(X_test)
#     test_accuracy = accuracy_score(y_test, test_predictions)

#     return val_accuracy, test_accuracy

# main function

> - 整理執行時出現的問題

> 1. 這裡我先假設了 y 是類別型

> 2. 發現 X 有 NaN值，所以增加了處理方式<br></br>
 → 加上一個 function : handle_missing_values

> 3. 即使在代碼中沒有明確傳遞樣本權重，BaggingClassifier 仍會嘗試傳遞樣本權重給基分類器。對於不支持樣本權重的分類器，如 KNeighborsClassifier，這會引發錯誤。<br></br>
 → TypeError: Underlying estimator KNeighborsClassifier does not support sample weights.<br></br>
 → 修改 BaggingClassifier 的參數

> 4. X_train 和 y_train 是 numpy 數組，而不是 pandas 的 DataFrame 或 Series。<br></br>
 → 轉換成 pandas 的數據結構，或者直接使用 numpy 的索引，避免 AttributeError。<br></br>
 → 這裡是使用 numpy 索引的版本。

> 5. 在 train_and_evaluate_bagging_clf 函數中，將 y_train 轉換為 numpy 數組，避免 KeyError。<br></br>
 → 將 y_train 轉換為 numpy 數組

> 6. BaggingClassifier 在訓練之前不會創建 estimators_ 屬性，因此在 fit 方法調用之前不能訪問該屬性

> 7. 你的 VotingClassifier 中的一個或多個估計器的 predict_proba 方法傳回結果的形狀不一致，這可能是因為不同的估計器預測了不同數量的類別。錯誤訊息指出在 2 維之後形狀不一致，進一步支持了這一點
 → 處理之後還是有這個問題，表示你的個別估計器的訓練或配置方式存在更深層的問題。

In [479]:
def main():
    # 讀取資料
    df = load_data('/content/sample_data/df_final_with_eco.csv')
    if df is None:
        return

    # 切割資料集
    df_eco, df_near_estate = split_datasets(df)

    # 設置資料和標籤
    X, y = prepare_data(df_near_estate)

    # 處理缺失值
    X = handle_missing_values(X)

    # # 確定訓練集和測試集的大小
    # n = len(df_near_estate)
    # n_train = int(0.8 * n)
    # n_test = n - n_train
    # # 切割訓練集和測試集
    # X_train = X[:n_train]
    # y_train = y[:n_train]
    # X_test = X[-n_test:]
    # y_test = y[-n_test:]

    # 切割資料集成訓練集、測試集和驗證集
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)  # 70% 訓練集，30% 暫存集
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)  # 15% 測試集，15% 驗證集

    # 創建袋裝分類器
    bagging_classifiers, classifiers = create_bagging_classifiers()

    # 訓練袋裝分類器
    bagging_classifiers = train_bagging_classifiers(bagging_classifiers, X_train, y_train)

    # 計算袋裝分類器的預測結果
    bagging_predictions = make_predictions(bagging_classifiers, X_test, y_test)

    # 計算袋裝分類器的準確率
    bagging_accuracies = calculate_accuracies(bagging_predictions, y_test)

    # 打印袋裝分類器的準確率
    for name, accuracy in bagging_accuracies.items():
        print(f"{name} Bagging Accuracy: {accuracy}")

    # 訓練基分類器
    for name, clf in classifiers.items():
        clf.fit(X_train, y_train)

    # 創建投票分類器
    voting_clf = create_voting_classifier(classifiers, X_train)

    # 創建袋裝投票分類器
    bagging_clf = create_bagging_classifier(voting_clf)

    # 訓練並評估袋裝投票分類器
    # ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (5, 758) + inhomogeneous part.
    # 可能表示你的個別估計器的訓練或配置方式存在更深層的問題。
    val_accuracy, test_accuracy = train_and_evaluate_bagging_clf(bagging_clf, X_train, y_train, X_val, y_val, X_test, y_test)

    # 打印準確率
    print("Validation Accuracy: ", val_accuracy)
    print("Test Accuracy: ", test_accuracy)

# 執行主函數
main()

Training KNN
Training XGBoost
Training Neural Network
Training SVM
Training Random Forest
Available classifiers in base_clfs: dict_keys(['KNN', 'XGBoost', 'Neural Network', 'SVM', 'Random Forest'])
KNN predict_proba shape: (10, 9)
XGBoost predict_proba shape: (10, 9)
Neural Network predict_proba shape: (10, 9)
SVM predict_proba shape: (10, 9)
Random Forest predict_proba shape: (10, 9)


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


Estimator VotingClassifier raised ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (5, 1) + inhomogeneous part.. Retraining on full data...
Estimator VotingClassifier raised ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (5, 1) + inhomogeneous part.. Retraining on full data...
Estimator VotingClassifier raised ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (5, 1) + inhomogeneous part.. Retraining on full data...
Estimator VotingClassifier raised ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (5, 1) + inhomogeneous part.. Retraining on full data...
Estimator VotingClassifier raised ValueError: setting an arr

IndexError: index 8 is out of bounds for axis 1 with size 8